First create some helper functions to generate the data etc

In [1]:
library(e1071)
set.seed(1)
x=seq(1:400)
train=sample(1:400,200)

genData = function(w,x,b,noise){
    
    return (((w %*% x) + b + noise*runif(n=length(x),min=-1,max=1))[1,])
    
}

Warning message:
"package 'e1071' was built under R version 3.3.3"

This function will show the data and also an envelope around the noise (but not around spikes)

In [2]:
plotData=function(y,w,x,b,noise){
    plot(x,y)
    abline(b,w,col="green",lty="dashed")
    abline(b+(noise),w,col="blue",lty="dashed")
    abline(b-(noise),w,col="blue",lty="dashed")
}

Helper functions to run the LM/SVR/Tuned SVR and also to calculte the MSE

In [3]:
lmcaller=function(y,x,train){
    df=as.data.frame(cbind(x,y))    
    ytest = y[-train]
    model=lm(y~x,data=df,subset=train)
    yhat=predict(model,newdata=df[-train,])    
    list(coef = model$coef,MSE=MSE(ytest,yhat),yhat=yhat)
}

svrcaller=function(y,x,train){
    df=as.data.frame(cbind(x,y))    
    ytest = y[-train]
    model=svm(y~x,data=df,subset=train)
    yhat=predict(model,newdata=df[-train,])    
    list(MSE=MSE(ytest,yhat),yhat=yhat)
}

tunedSvrCaller=function(y,x,train){
    df=as.data.frame(cbind(x,y))
    ytest = y[-train]
    xtest = x[-train]
    tuner=tune(svm,y~x,data=df[train,],
                  ranges=list(epsilon=seq(0,0.2,0.01),cost=2^(2:5)))
    
    # tuner can be plotted using the plot(tuner) function. But I won't do that now.
    
    tunedModel = tuner$best.model
    yhat=predict(tunedModel,newdata=df[-train,])
    list(MSE=MSE(ytest,yhat),yhat=yhat)
}

MSE=function(y,yhat){
    sqrt(mean((yhat-y)^2))
}

A helper function to run the regressions and plot the data and errors 

In [4]:
runRegression=function(y,w,x,b,noise,train,title){
    
    ytest=y[-train]
    xtest=x[-train]

    lr=lmcaller(y,x,train)
    svr=svrcaller(y,x,train)    
    tsvr=tunedSvrCaller(y,x,train)
    
    results = list(
        MSE=as.vector(cbind(
            LR_MSE=lr$MSE,
            SVR_MSE=svr$MSE,
            TSVR_MSE=tsvr$MSE
        )),
        LM_COEF=lr$coef
    )
    
    minMSE = which.min(results$MSE)
    
    #par(mfrow=c(2,2))
    plotData(y,w,x,b,noise)
    title(title)
    barplot(results$MSE,
        col=ifelse(1:3 == minMSE,"green","red"),
        names=c("LM","SVR","Tuned SVR")
    )
    
}

Now lets create the data. Let's begin by simulating some data that satisfies $$y=w\cdot x + b + \epsilon$$

Where $w=1$ and $b=10$

##### Case1: A moderate amount of noise, no spikes

In [5]:
case1 = function(){    
    w=1
    b=10
    noise=15
    y=genData(w,x,b,noise)
    runRegression(y,w,x,b,noise,train,"Case1")
}

##### Case2: A large amount of noise, no spikes

In [6]:
case2 = function(){    
    w=1
    b=10
    noise=50
    y=genData(w,x,b,noise)
    runRegression(y,w,x,b,noise,train,"Case2")
}

###### Case3: A very small amount of noise, no spikes

In [7]:
case3 = function(){    
    w=1
    b=10
    noise=0.5
    y=genData(w,x,b,noise)
    runRegression(y,w,x,b,noise,train,"Case3")
}

##### Case4: Moderate noise, 10% of data has some spikes

In [8]:
case4 = function(){    
    w=1
    b=10
    noise=15
    y=genData(w,x,b,noise)
    
    #Now add some spikes at certain indices 
    indices=sample(1:length(y),length(y)/10)
    y[indices]=y[indices]+ 100*runif(length(indices),min=-1,max=1)
    runRegression(y,w,x,b,noise,train,"Case4")
}

##### Case5: Moderate noise, 5% of data has big spikes

In [9]:
case5 = function(){    
    w=1
    b=10
    noise=15
    y=genData(w,x,b,noise)
    
    #Now add even more extra spiky obs, every 5th record will be a spike
    indices=sample(1:length(y),length(y)/5)
    y[indices]=y[indices]+ 100*runif(length(indices),min=-1,max=1)
    runRegression(y,w,x,b,noise,train,"Case5")
}

##### Case6: Moderate noise, 5% of data has huge spikes

In [10]:
case6 = function(){    
    w=1
    b=10
    noise=15
    y=genData(w,x,b,noise)
    
    #Now add even more extra spiky obs, every 5th record will be a spike, and the size of the spikes is bigger 
    indices=sample(1:length(y),length(y)/5)
    y[indices]=y[indices]+ 500*runif(length(indices),min=-1,max=1)
    runRegression(y,w,x,b,noise,train,"Case6")
}

### Now run all the cases

In [ ]:
par(mfrow=c(2,2))
case1()
case2()
case3()
case4()
case5()
case6()